In [ ]:
# Установка необходимых зависимостей
!pip install ultralytics
!pip install torch torchvision
!pip install albumentations

# Настройка CUDA для лучшего управления памятью
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

# Клонирование репозитория с обученной моделью
!git clone https://github.com/wrjgnouwbugbw/Car_plate_detecting.git

# Добавляем путь к репозиторию в PYTHONPATH
import sys
sys.path.append('/kaggle/working/Car_plate_detecting')

In [ ]:
# Подготовка данных для дообучения
import shutil
import os

# Создаем директории для данных дообучения
!mkdir -p /kaggle/working/finetune_dataset/train/images
!mkdir -p /kaggle/working/finetune_dataset/train/labels
!mkdir -p /kaggle/working/finetune_dataset/valid/images
!mkdir -p /kaggle/working/finetune_dataset/valid/labels

# Копируем только оригинальный датасет
!cp /kaggle/working/dataset/train/images/* /kaggle/working/finetune_dataset/train/images/
!cp /kaggle/working/dataset/train/labels/* /kaggle/working/finetune_dataset/train/labels/
!cp /kaggle/working/dataset/val/images/* /kaggle/working/finetune_dataset/valid/images/
!cp /kaggle/working/dataset/val/labels/* /kaggle/working/finetune_dataset/valid/labels/

# Создаем отдельную директорию для тестирования аугментаций на сложных случаях
!mkdir -p /kaggle/working/hard_cases_test/images
!cp -r /kaggle/working/Car_plate_detecting/hard_cases/* /kaggle/working/hard_cases_test/images/

print("Количество изображений для дообучения:", len(os.listdir('/kaggle/working/finetune_dataset/train/images')))
print("Количество изображений для валидации:", len(os.listdir('/kaggle/working/finetune_dataset/valid/images')))
print("Количество сложных случаев для тестирования:", len(os.listdir('/kaggle/working/hard_cases_test/images')))

In [ ]:
# Настройка конфигурации для дообучения
import configparser
from pathlib import Path

# Загрузка базовой конфигурации
config = configparser.ConfigParser()
config.read("/kaggle/working/Car_plate_detecting/Config/Config.ini")

# Модификация настроек для дообучения
train_config = config["Train"]

# Путь к предобученной модели
train_config["weights"] = "/kaggle/working/Car_plate_detecting/best.pt"

# Пути к датасету
train_config["train"] = "/kaggle/working/finetune_dataset/train"
train_config["val"] = "/kaggle/working/finetune_dataset/valid"

# Настройка аугментаций для решения выявленных проблем
train_config["degrees"] = "10.0"     # Небольшой поворот
train_config["scale"] = "0.5"        # Масштабирование
train_config["perspective"] = "0.001" # Легкая перспектива
train_config["flipud"] = "0.3"       # Разрешаем вертикальный переворот
train_config["fliplr"] = "0.3"       # Разрешаем горизонтальный переворот
train_config["mosaic"] = "1.0"       # Включаем мозаику
train_config["mixup"] = "0.3"        # Включаем mixup

# Параметры для быстрого дообучения
train_config["epochs"] = "15"        # Немного увеличиваем эпохи для лучшей адаптации
train_config["batch_size"] = "32"    # Сохраняем большой batch size
train_config["patience"] = "7"       # Увеличиваем patience для лучшей сходимости
train_config["optimizer"] = "AdamW"  # Используем AdamW для дообучения
train_config["lr0"] = "0.0001"      # Небольшой learning rate для тонкой настройки

# Сохраняем обновленную конфигурацию
with open('/kaggle/working/Car_plate_detecting/Config/Config_finetune.ini', 'w') as configfile:
    config.write(configfile)

In [ ]:
# Функция для тестирования аугментаций на сложных случаях
from ultralytics import YOLO
import cv2

def test_on_hard_cases(model_path, hard_cases_dir):
    model = YOLO(model_path)
    
    success_count = 0
    total_cases = 0
    
    print("\nТестирование на сложных случаях:")
    print("=" * 50)
    
    # Проходим по всем сложным случаям
    for img_file in os.listdir(hard_cases_dir):
        if img_file.endswith(('.jpg', '.jpeg', '.png', '.bmp')):
            total_cases += 1
            img_path = os.path.join(hard_cases_dir, img_file)
            
            # Пытаемся детектировать номер
            results = model.predict(img_path, conf=0.25)
            
            # Если есть хоть одна детекция, считаем успехом
            if len(results[0].boxes) > 0:
                success_count += 1
                print(f"✅ {img_file}: Номер обнаружен")
            else:
                print(f"❌ {img_file}: Номер не обнаружен")
    
    success_rate = (success_count / total_cases) * 100 if total_cases > 0 else 0
    print("\nРезультаты:")
    print(f"Всего сложных случаев: {total_cases}")
    print(f"Успешных детекций: {success_count}")
    print(f"Процент успеха: {success_rate:.2f}%")
    
    return success_rate

# Будем проверять модель на сложных случаях после каждой эпохи
from ultralytics.callbacks import on_train_epoch_end

def on_epoch_end(trainer):
    # Проверяем модель на сложных случаях
    current_model_path = trainer.best if trainer.best else trainer.last
    success_rate = test_on_hard_cases(current_model_path, '/kaggle/working/hard_cases_test/images')
    print(f"\nЭпоха {trainer.epoch}: Точность на сложных случаях = {success_rate:.2f}%")

In [ ]:
# Запускаем процесс дообучения
from yolo9_train import train

# Загружаем конфигурацию для дообучения
config = configparser.ConfigParser()
config.read("/kaggle/working/Car_plate_detecting/Config/Config_finetune.ini")

# Тестируем исходную модель на сложных случаях
print("Тестирование исходной модели:")
initial_success_rate = test_on_hard_cases("/kaggle/working/Car_plate_detecting/best.pt", 
                                        '/kaggle/working/hard_cases_test/images')

# Запускаем дообучение с проверкой на сложных случаях после каждой эпохи
train(config["Train"], callbacks=[on_epoch_end])

# Тестируем финальную модель на сложных случаях
print("\nТестирование после дообучения:")
final_success_rate = test_on_hard_cases("/kaggle/working/Car_plate_detecting/best.pt", 
                                      '/kaggle/working/hard_cases_test/images')

print(f"\nУлучшение на сложных случаях: {final_success_rate - initial_success_rate:.2f}%")